<h1>Sentiment Analysis project<h1>
<h3>Name student1: Tego Calderon<h3>
<h3>Name student2: Benito Antonio M.<h3>

In [60]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# One-time setup steps
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


#--------Data Wrangling--------
#Step 1: Read the data
file_path = "../dataset/customer_review.tsv"
df = pd.read_csv(file_path, sep='\t')

#Step 2: Print the first 10 lines
print("Printing first 10 rows of customer_review.tsv")
print(df.head(10))

#Step 3: Verify that data does not contain missing values
print("\nChecking for missing values:")
print(df.isnull().sum())

#Step 4: Remove stop words
def remove_stopwords(reviews):
    words = nltk.word_tokenize(reviews)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['cleaned_reviews'] = df['verified_reviews'].apply(remove_stopwords)





[nltk_data] Downloading package stopwords to /Users/dbhoo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Printing first 10 rows of customer_review.tsv
   rating       date             variation  \
0       5  31-Jul-18      Charcoal Fabric    
1       5  31-Jul-18      Charcoal Fabric    
2       4  31-Jul-18        Walnut Finish    
3       5  31-Jul-18      Charcoal Fabric    
4       5  31-Jul-18      Charcoal Fabric    
5       5  31-Jul-18  Heather Gray Fabric    
6       3  31-Jul-18     Sandstone Fabric    
7       5  31-Jul-18      Charcoal Fabric    
8       5  30-Jul-18  Heather Gray Fabric    
9       5  30-Jul-18  Heather Gray Fabric    

                                    verified_reviews  feedback  
0                                      Love my Echo!         1  
1                                          Loved it!         1  
2  Sometimes while playing a game, you can answer...         1  
3  I have had a lot of fun with this thing. My 4 ...         1  
4                                              Music         1  
5  I received the echo as a gift. I needed anothe...     

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/dbhoo/nltk_data'
    - '/Users/dbhoo/github-classroom/COP4023-fall23/finalproject-plb/.venv/nltk_data'
    - '/Users/dbhoo/github-classroom/COP4023-fall23/finalproject-plb/.venv/share/nltk_data'
    - '/Users/dbhoo/github-classroom/COP4023-fall23/finalproject-plb/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
